In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""

@author: in2293
"""

import os
import matplotlib.pyplot as plt
import numpy as np


from analyzer_mod import analyzer
from wf_analysis_mod import wf_analyzer

In [2]:
# Pick an animal and experiment

#global AUE ACQinfo Analyzer datadir
#global Analyzer

#anim = 'xt7'
#expt = 'u009_004'; #epi ret expt
#ISIflag = 1


#anim = 'xt8'
#expt = 'u003_009' 
#ISIflag = 1
#pixpermm = 79 

anim = 'yd3'
expt = 'u001_002'
pixpermm = 79
ISIflag = 0

In [3]:
#Create the analyzer object. Uses analyzer file.

rootanadir = '/Volumes/TOSHIBA EXT/AnalyzerFiles/'

anafname = anim + '_' + expt + '.analyzer'
anapath = os.path.join(*[rootanadir, anim , anafname])

analyzerX = analyzer(anapath) 

In [4]:
#Create the widefield analysis object. Uses data file.

rootdatadir = '/Volumes/TOSHIBA EXT/ISIData/'

datapath = os.path.join(*[rootdatadir,anim,expt])  #Just the folder. Each frame is a separate file.

wfX = wf_analyzer(datapath,analyzerX)  

Data successfully loaded.


In [5]:
#%% Set trial averaging window

if ISIflag:
    baselineWin = [-500, 500] #ms [start, stop]
    responseWin = [1000,  analyzerX.P['stim_time']*1000+2000]


else:   #GCaMP
    baselineWin = [-500, 0] #ms [start, stop]
    responseWin = [100, analyzerX.P['stim_time']*1000+500]

In [6]:
# Get a baseline and response image for each trial

RMat = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.ntrials));
BMat = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.ntrials));

for tno in range(analyzerX.ntrials):  #loop trials
    
    print(tno)
    Rxt = wfX.loadtrial(tno,responseWin,analyzerX)
    Bxt = wfX.loadtrial(tno,baselineWin,analyzerX)
    
    if ISIflag:              #Invert, b/c ISI is a negative signal.
        bitdepth = 16;  #Pco Panda
        Rxt = 2^bitdepth - Rxt;
  
        
    RMat[:,:,tno] = np.mean(Rxt,axis = 2); #Store the mean response image of this trial
    BMat[:,:,tno] = np.mean(Bxt,axis = 2); #Store the mean baseline image of this trial


0
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
1
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
2
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
3
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
4
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
5
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
6
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
7
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
8
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
9
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
10
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
11
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
12
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
13
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
14
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
15
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
16
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
17
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
18
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
19
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
20
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
21
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
22
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
23
(12, 80)
[100, 3500]
(0, 10)
[-500, 0]
24

In [ ]:
#Baseline normalization

#RMat2 = RMat2
#RMat2 = (RMat-BMat) + mean(BMat,3)
RMat2 = (RMat-BMat)/BMat

In [ ]:
# Organize the images by condition number and take mean over the repeats.  
# We are left with one image from each condition after this

RMat3 = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.nconditions))
BMat3 = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.nconditions))

for c in range(analyzerX.nconditions):
    
    nr = analyzerX.nrepeats(c);
  
    tnos = np.zeros(nr).astype(int)
    for r in range(nr):
        tnos[r] = analyzerX.trialno(c,r)

    RMat3[:,:,c] = np.mean(RMat2[:,:,tnos],axis = 2)  #mean over repeats
    BMat3[:,:,c] = np.mean(BMat[:,:,tnos],axis = 2)
    
    
blankflag = 0
if analyzerX.loops['conds'][-1]['symbol'][0] == 'blank':
    blankflag = 1

In [ ]:
# All the code up until this point could be applied to any experiment.  
# Here is where it starts to assume certain looper variable 

#get the index within the looper
for p in range(len(analyzerX.L['param'])):
    if analyzerX.L['param'][p][0] == 's_phase2':
        phaseDim = p

    if analyzerX.L['param'][p][0] == 'ori2':
        oriDim = p

#valmat is a matrix containing the looper values for each condition

valMat = np.zeros((len(analyzerX.loops['conds'][c]['val']),analyzerX.nconditions-blankflag))
                  
for c in range(analyzerX.nconditions-blankflag):
    for p in range(len(analyzerX.loops['conds'][c]['val'])):
        valMat[p,c] = analyzerX.loops['conds'][c]['val'][p]
        


oridom = list(set(valMat[oriDim,:]))
phasedom = list(set(valMat[phaseDim,:]))


In [ ]:
# Plot mean response to each condition


plt.figure()
k = 1
for ori in range(len(oridom)):
    for p in range(len(phasedom)):

        a = np.where(valMat[oriDim,:] == oridom[ori])[0]
        b = np.where(valMat[phaseDim,:] == phasedom[p])[0]
        id = np.intersect1d(a,b)[0]
        
        imdum = RMat3[:,:,id];
        
        plt.subplot(len(oridom),len(phasedom),k)
  
        
        plt.imshow(imdum*bw)
        
        k += 1